In [1]:
from keras.datasets import reuters
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from keras import preprocessing
from keras import backend as K
from math import pi, floor
import random
from codeepneat import codeepneat, config, population, chromosome, genome, visualize
import pickle
import numpy as np
import keras
import collections
import sys

Using TensorFlow backend.


In [2]:
path = keras.utils.get_file(
    'ptb.train.txt',
    origin='https://raw.githubusercontent.com/wojzaremba/lstm/master/data/ptb.train.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))



Corpus length: 5101618


In [3]:
path = keras.utils.get_file(
    'ptb.valid.txt',
    origin='https://github.com/wojzaremba/lstm/blob/master/data/ptb.valid.txt')
text_val = open(path).read().lower()
print('Corpus length:', len(text))


Corpus length: 5101618


In [4]:
# Length of extracted character sequences
maxlen = 100

# We sample a new sequence every `step` characters
step = 2

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x_train = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y_train = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_train[i, t, char_indices[char]] = 1
    y_train[i, char_indices[next_chars[i]]] = 1

Number of sequences: 2550759
Unique characters: 49
Vectorization...


In [5]:
# Length of extracted character sequences
maxlen = 100

# We sample a new sequence every `step` characters
step = 2

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars_val = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices_val = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x_val = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y_val = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_val[i, t, char_indices[char]] = 1
    y_val[i, char_indices[next_chars[i]]] = 1

Number of sequences: 2550759
Unique characters: 49
Vectorization...


In [6]:
data = [x_train[0:100], y_train[0:100], x_val[0:100], y_val[0:100]]

In [7]:
%%file configReuters
#--- parameters for the robot experiment ---#
[phenotype]
input_nodes         = 100, 49
output_nodes        = 49
conv                = False
LSTM                = True

[genetic]
max_fitness_threshold = 1

# Human reasoning
pop_size              = 10
prob_addconv          = 0.0
prob_addLSTM          = 1
prob_addlayer         = 0.1
prob_mutatelayer      = 0.4
prob_addmodule        = 0.05
prob_switchmodule     = 0.3
elitism               = 1

[genotype compatibility]
compatibility_threshold = 3.0
compatibility_change    = 0.0
excess_coefficient      = 5.0
disjoint_coefficient    = 3.0
connection_coefficient  = 0.4
size_coefficient        = 0.8

[species]
species_size        = 10
survival_threshold  = 0.2
old_threshold       = 30
youth_threshold     = 10
old_penalty         = 0.2
youth_boost         = 1.2
max_stagnation      = 15

Overwriting configReuters


In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [9]:
def fitness(network, data):
    K.set_learning_phase(1)
    network.fit(data[0], data[1],  epochs=20, batch_size = 128)
    K.set_learning_phase(0)
    start_index = random.randint(0, 2550658)
    generated_text = text_val[start_index: start_index + 100]
    for i in range(400):
        sampled = np.zeros((1, 100, 49))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices_val[char]] = 1.

        preds = network.predict(sampled, verbose=0)[0]
        next_index = sample(preds, 0.2)
        next_char = chars_val[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()
        
 
    #return acc

In [10]:
def evolve(n, debugging=False):
    if(debugging):
        debug = open("debug.txt", "w")
    else:
        debug = None
    config.load('configReuters')
    # Create 2 separate populations (size is now defined explicitly, but config file can still be used)
    module_pop = population.Population(15, chromosome.ModuleChromo, debug=debug)
    # As the top hierarchical level, the blueprint population needs to be able to see the module population
    blueprint_pop = population.Population(10, chromosome.BlueprintChromo, module_pop, debug=debug)
    # Most of the actual evolving is now handled outside of the population, by CoDeepNEAT
    # Instead of requiring the user to overwrite the evaluation function, CoDeepNEAT evaluates the populations itself,
    # it simply requires a fitness function for the networks it creates passed in as an argument.
    codeepneat.epoch(n, blueprint_pop, module_pop, 2, fitness, data, save_best=True, name='reuters', debug=debug)
    # It will still stop if fitness surpasses the max_fitness_threshold in config file
    # Plots the evolution of the best/average fitness
    visualize.plot_stats(module_pop.stats, name="ptbmod_")
    visualize.plot_stats(blueprint_pop.stats, name="bp_")
    # Visualizes speciation
    #visualize.plot_species(module_pop.species_log, name="NMISTmod_")
    #visualize.plot_species(blueprint_pop.species_log, name="NMISTbp_")

In [11]:
evolve(25, True)

-----Generation 0--------
Network 0
Epoch 1/20
100/100 [==============================] - 1s - loss: 3.8899 - acc: 0.0300
Epoch 2/20
100/100 [==============================] - 0s - loss: 3.8102 - acc: 0.2100
Epoch 3/20
100/100 [==============================] - 0s - loss: 3.4022 - acc: 0.2100
Epoch 4/20
100/100 [==============================] - 0s - loss: 3.4783 - acc: 0.0900
Epoch 5/20
100/100 [==============================] - 0s - loss: 3.0866 - acc: 0.0900
Epoch 6/20
100/100 [==============================] - 0s - loss: 2.9999 - acc: 0.2100
Epoch 7/20
100/100 [==============================] - 0s - loss: 2.9720 - acc: 0.2100
Epoch 8/20
100/100 [==============================] - 0s - loss: 2.9571 - acc: 0.2100
Epoch 9/20
100/100 [==============================] - 0s - loss: 2.9478 - acc: 0.2100
Epoch 10/20
100/100 [==============================] - 0s - loss: 2.9409 - acc: 0.2100
Epoch 11/20
100/100 [==============================] - 0s - loss: 2.9356 - acc: 0.2100
Epoch 12/20
100/

KeyError: '='